In [5]:
import pyart
import os
# from netCDF4 import num2date


In [6]:
def get_alt_lon_lat(radar):
    """
    :param radar: radar object
    :return: max and min limits of radar grid (x, y, z)
    """
    min_ix = pyart.core.Radar.get_gate_x_y_z(radar, 0)[0].min().astype('int')
    max_ix = pyart.core.Radar.get_gate_x_y_z(radar, 0)[0].max().astype('int')
    min_iy = pyart.core.Radar.get_gate_x_y_z(radar, 0)[1].min().astype('int')
    max_iy = pyart.core.Radar.get_gate_x_y_z(radar, 0)[1].max().astype('int')
    min_iz = pyart.core.Radar.get_gate_x_y_z(radar, 0)[2].min().astype('int')
    max_iz = pyart.core.Radar.get_gate_x_y_z(radar, 0)[2].max().astype('int')
    return min_ix, max_ix, min_iy, max_iy, min_iz, max_iz

In [7]:
def rain_tiff(radar_object):
    """
    :type radar_object: radar object
    :param radar_object: path to radar files
    :return: radar rainfall rate geotiff
    """

    radar = radar_object
    try:
        a = radar.metadata['instrument_name'].decode('utf-8')
    except AttributeError:
        a = radar.metadata['instrument_name']

    var2filter = 'cross_correlation_coefficient'
    thres_filter = 0.8
    var2plot = 'reflectivity'

    min_ix, max_ix, min_iy, max_iy, min_iz, max_iz = get_alt_lon_lat(radar)
    shape_xy = (max_ix * 2 / (radar.range['data'][1] - radar.range['data'][0])).astype('int')
    shape_z = (max_iz / 500).astype('int')

    rain_z = pyart.retrieve.est_rain_rate_z(radar, refl_field=var2plot, rr_field='rain_z')
    radar.add_field('rain_z', rain_z)

    grid = pyart.map.grid_from_radars((radar,), gatefilters=None,
                                      grid_shape=(shape_z + 1, shape_xy + 1, shape_xy + 1),
                                      grid_limits=((min_iz, max_iz), (min_ix, max_ix), (min_iy, max_iy)),
                                      fields=['rain_z'])

    folder_save = './'

    if os.path.exists(folder_save):
        pass
    else:
        make_dir(folder_save)

    name_save = 'radar_rain_st'
    file_save = '{0}/{1}.tif'.format(folder_save, name_save)

    pyart.io.write_grid_geotiff(grid, file_save, 'rain_z', warp=True, use_doublequotes=True)


In [8]:
radar = pyart.io.read('./data/CAR190917171051.RAW6KYV')
rain_tiff(radar)

C:\ProgramData\Anaconda3\lib\site-packages\pyart\io\sigmet.py:147: RuntimeWarning: invalid value encountered in sqrt
  sigmet_data, sigmet_metadata = sigmetfile.read_data(full_xhdr=full_xhdr)
C:\ProgramData\Anaconda3\lib\site-packages\pyart\map\gates_to_grid.py:177: DeprecationWarning: Barnes weighting function is deprecated. Please use Barnes 2 to be consistent with Pauley and Wu 1990.
  " Pauley and Wu 1990.", DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\pyart\filters\gatefilter.py:626: RuntimeWarning: invalid value encountered in less
  marked = self._get_fdata(field) < value


MissingOptionalDependency: GDAL not detected, GeoTIFF output failure!